# Computational Neuroscience Final Project
## Elizabeth Frey

In [71]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import Data

In [137]:
import pandas as pd
import numpy as np

# read csv file
df = pd.read_csv('data-en-hi-de-fr.csv')

In [138]:
from tensorflow.keras.utils import to_categorical

# create a dictionary of the languages
languages = {'en': 0, 'hi': 1, 'de': 2, 'fr': 3}

# group the dataframe by labels column
grouped = df.groupby('labels')

# create two dataframes, one for ham and one for spam
original_df = df
ham_df = grouped.get_group('ham')
spam_df = grouped.get_group('spam')


def processDf(df):
    df.drop('labels', axis=1, inplace=True)
    df.rename(columns={'text': 'en', 'text_hi': 'hi',
              'text_de': 'de', 'text_fr': 'fr'}, inplace=True)
    df = df.melt(var_name='language', value_name='phrase')
    df['language_code'] = df['language'].map(languages)
    return df


original_df = processDf(original_df)
ham_df = processDf(ham_df)
spam_df = processDf(spam_df)

# ham_df = ham_df.drop('labels', axis=1)
# spam_df = spam_df.drop('labels', axis=1)

# # pivot the data so that each row is a phrase in a language
# ham_df = ham_df.rename(
#     columns={'text': 'en', 'text_hi': 'hi', 'text_de': 'de', 'text_fr': 'fr'})
# ham_df = ham_df.melt(var_name='language', value_name='phrase')
# spam_df = spam_df.rename(
#     columns={'text': 'en', 'text_hi': 'hi', 'text_de': 'de', 'text_fr': 'fr'})
# spam_df = spam_df.melt(var_name='language', value_name='phrase')


# # convert the languages to numbers
# ham_df['language_code'] = ham_df['language'].map(languages)
# spam_df['language_code'] = spam_df['language'].map(languages)

/var/folders/dm/l18067jn62qb3fkpng93sqhr0000gn/T/ipykernel_15227/3097920759.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('labels', axis=1, inplace=True)
/var/folders/dm/l18067jn62qb3fkpng93sqhr0000gn/T/ipykernel_15227/3097920759.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'text': 'en', 'text_hi': 'hi',
/var/folders/dm/l18067jn62qb3fkpng93sqhr0000gn/T/ipykernel_15227/3097920759.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

## Creating Functions

### Embeddings

In [145]:

# define a function to tokenize the phrases in each dataframe

def tokenizeDf(df):
    tokens = [phrase.split() for phrase in df['phrase']]
    df['tokens'] = tokens
    return df

#### Word2Vec Embeddings

In [141]:
import gensim
from gensim.models import Word2Vec


def addWord2VecEmbeddings(df):
    # train the word2vec model on the ham and spam tokens
    model = Word2Vec(df['tokens'], min_count=1)
    # create the embeddings for each dataframe
    df['w2v_embedding'] = df['tokens'].apply(
        lambda x: model.wv[x].mean(axis=0).astype(np.float32))
    return df

### Train the Model

#### Create Training and Testing Data Functions

In [121]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

listOfEmbeddings = ['w2v_embedding']

# Create Training and Testing Data Functions


def createTrainingTestingData(embeddingType, df):
    if embeddingType not in listOfEmbeddings:
        print('Invalid embedding type')
        return

    X = df[embeddingType]
    y = df['language_code']

    # create the training and testing dataframes
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

    y_train_one_hot = to_categorical(y_train, 4)
    y_test_one_hot = to_categorical(y_test, 4)

    # convert the training and testing dataframes to numpy arrays
    X_train = np.array([x for x in X_train])
    X_test = np.array([x for x in X_test])
    y_train = np.array([y for y in y_train_one_hot])
    y_test = np.array([y for y in y_test_one_hot])

    # reshape the training and testing dataframes
    X_train = X_train.reshape(X_train.shape[0], 1, 100)
    X_test = X_test.reshape(X_test.shape[0], 1, 100)

    # Assuming X_train and y_train are NumPy arrays
    X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
    X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
    y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

    return X_train, X_test, y_train, y_test

#### Create Model Function

In [95]:
from tensorflow import keras
from tensorflow.keras import layers


def createModel():
    model = keras.Sequential()
    model.add(layers.GRU(64, input_shape=(1, 100)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(units=4, activation='softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer="adam",
        metrics=["accuracy"],
    )
    return model

#### Fit the Model using Training Data Function

In [96]:
def trainModel(model, X_train, y_train):
    model.fit(
        X_train, y_train, batch_size=64, epochs=10
    )
    return model

### Test the Model

#### Function to print ten random predictions

In [97]:
def testModel(model, X_test, y_test):
    for i in range(10):
        result = tf.argmax(model.predict(tf.expand_dims(X_test[i], 0)), axis=1)
        print(result.numpy(), y_test[i])

### Evalute the Model

In [143]:
def evaluateModel(model, X_test, y_test):
    model.evaluate(X_test, y_test)

## Evaluate Model with Different Embeddings

In [142]:
def word2Vec(df):
    df = tokenizeDf(df)
    df = addWord2VecEmbeddings(df)
    X_train, X_test, y_train, y_test = createTrainingTestingData(
        'w2v_embedding', original_df)
    model = createModel()
    model.summary()
    model = trainModel(model, X_train, y_train)
    testModel(model, X_test, y_test)
    evaluateModel(model, X_test, y_test)


word2Vec(original_df)

  language                                             phrase  language_code  \
0       en  Go until jurong point, crazy.. Available only ...              0   
1       en                      Ok lar... Joking wif u oni...              0   
2       en  Free entry in 2 a wkly comp to win FA Cup fina...              0   
3       en  U dun say so early hor... U c already then say...              0   
4       en  Nah I don't think he goes to usf, he lives aro...              0   

                                              tokens  
0  [Go, until, jurong, point,, crazy.., Available...  
1               [Ok, lar..., Joking, wif, u, oni...]  
2  [Free, entry, in, 2, a, wkly, comp, to, win, F...  
3  [U, dun, say, so, early, hor..., U, c, already...  
4  [Nah, I, don't, think, he, goes, to, usf,, he,...  
Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_20 (GRU)                (None